# Optimization: Training Neural Networks

Optimization is the process of finding the best parameters (weights and biases) for our neural network. We use gradients computed by backpropagation to iteratively improve these parameters.

## What You'll Learn:
1. **Gradient Descent** - The fundamental algorithm
2. **Learning Rate** - The most important hyperparameter
3. **Momentum** - Accelerating training
4. **Adam** - The most popular optimizer
5. **Learning Rate Schedules** - Adapting over time
6. **Regularization** - Preventing overfitting

**Goal:** Minimize the loss function $L(\theta)$ where $\theta$ represents all parameters.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

np.random.seed(42)
torch.manual_seed(42)

## 1. Gradient Descent: The Foundation

**Gradient Descent** is the fundamental optimization algorithm. It updates parameters in the direction that decreases the loss.

### Algorithm:
Repeat until convergence:
1. Compute gradient: $g = \nabla_\theta L(\theta)$
2. Update parameters: $\theta \leftarrow \theta - \alpha g$

where $\alpha$ is the **learning rate**.

### Intuition:
- The gradient points **uphill** (direction of steepest increase)
- We go in the **opposite direction** to decrease loss
- Learning rate controls step size

### 1.1 Visualizing Gradient Descent

In [ ]:
# Simple 1D example: minimize f(x) = x^2
def f(x):
    return x**2

def df_dx(x):
    return 2*x

# Gradient descent
x = 10.0  # starting point
learning_rate = 0.1
history = [x]

for _ in range(20):
    gradient = df_dx(x)
    x = x - learning_rate * gradient
    history.append(x)

# Visualize
x_vals = np.linspace(-10, 10, 200)
y_vals = f(x_vals)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(x_vals, y_vals, 'b-', linewidth=2, label='f(x) = x²')
plt.plot(history, [f(x) for x in history], 'ro-', markersize=8, label='Gradient descent path')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Gradient Descent on f(x) = x²')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot([f(x) for x in history], 'ro-', markersize=6)
plt.xlabel('Iteration')
plt.ylabel('f(x)')
plt.title('Loss Decreasing Over Time')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Starting point: x = {history[0]:.4f}, f(x) = {f(history[0]):.4f}")
print(f"Final point: x = {history[-1]:.4f}, f(x) = {f(history[-1]):.4f}")

### 1.2 Gradient Descent in 2D

In [ ]:
# 2D example: minimize f(x, y) = x^2 + y^2
def f_2d(x, y):
    return x**2 + y**2

def gradient_2d(x, y):
    return np.array([2*x, 2*y])

# Gradient descent
pos = np.array([8.0, 6.0])  # starting point
learning_rate = 0.1
history = [pos.copy()]

for _ in range(50):
    grad = gradient_2d(pos[0], pos[1])
    pos = pos - learning_rate * grad
    history.append(pos.copy())

history = np.array(history)

# Visualize
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(x, y)
Z = f_2d(X, Y)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
contour = plt.contour(X, Y, Z, levels=20)
plt.clabel(contour, inline=True, fontsize=8)
plt.plot(history[:, 0], history[:, 1], 'ro-', markersize=6, linewidth=2, label='GD path')
plt.plot(history[0, 0], history[0, 1], 'gs', markersize=12, label='Start')
plt.plot(history[-1, 0], history[-1, 1], 'r*', markersize=15, label='End')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Gradient Descent Path')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
losses = [f_2d(h[0], h[1]) for h in history]
plt.plot(losses, 'b-', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss Over Time')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Learning Rate: The Most Important Hyperparameter

The learning rate $\alpha$ controls how big a step we take.

### Issues:
- **Too small**: Slow convergence, might not reach minimum
- **Too large**: Oscillation, divergence, overshooting
- **Just right**: Fast, stable convergence

In [ ]:
def gradient_descent_1d(x0, learning_rate, num_steps=50):
    """Run gradient descent on f(x) = x^2"""
    x = x0
    history = [x]
    
    for _ in range(num_steps):
        grad = 2 * x
        x = x - learning_rate * grad
        history.append(x)
        
        # Stop if diverging
        if abs(x) > 1000:
            break
    
    return history

# Try different learning rates
x0 = 10.0
learning_rates = [0.01, 0.1, 0.5, 0.9, 1.1]

plt.figure(figsize=(14, 4))

for i, lr in enumerate(learning_rates, 1):
    history = gradient_descent_1d(x0, lr, num_steps=20)
    
    plt.subplot(1, len(learning_rates), i)
    plt.plot([f(x) for x in history], 'o-', markersize=4)
    plt.xlabel('Iteration')
    plt.ylabel('f(x)')
    plt.title(f'LR = {lr}')
    plt.grid(True, alpha=0.3)
    
    if len(history) > 1 and abs(history[-1]) > 100:
        plt.ylim(0, min(1000, max([f(x) for x in history[:10]])))

plt.tight_layout()
plt.show()

print("Observations:")
print("- LR=0.01: Very slow convergence")
print("- LR=0.1: Good convergence")
print("- LR=0.5: Fast but some oscillation")
print("- LR=0.9: Large oscillations")
print("- LR=1.1: Diverges! (explodes)")

## 3. Types of Gradient Descent

### 3.1 Batch Gradient Descent
- Uses **entire dataset** to compute gradient
- Accurate but slow for large datasets
- Formula: $\theta \leftarrow \theta - \alpha \frac{1}{m} \sum_{i=1}^{m} \nabla L^{(i)}$

### 3.2 Stochastic Gradient Descent (SGD)
- Uses **one sample** at a time
- Fast but noisy updates
- Formula: $\theta \leftarrow \theta - \alpha \nabla L^{(i)}$

### 3.3 Mini-Batch Gradient Descent (Most Common)
- Uses **small batches** (e.g., 32, 64, 128 samples)
- Balance of speed and stability
- Formula: $\theta \leftarrow \theta - \alpha \frac{1}{b} \sum_{i \in \text{batch}} \nabla L^{(i)}$

## 4. Momentum: Accelerating Gradient Descent

**Problem with vanilla GD:** Can be slow in ravines (steep in one direction, shallow in another)

**Momentum** adds a "velocity" term that accumulates gradients:

$v_t = \beta v_{t-1} + (1-\beta) g_t$

$\theta_{t+1} = \theta_t - \alpha v_t$

where:
- $v_t$ is the velocity
- $\beta$ is the momentum coefficient (typically 0.9)
- $g_t$ is the gradient

**Benefits:**
- Accelerates in consistent directions
- Dampens oscillations
- Helps escape local minima

In [ ]:
# Compare vanilla GD vs. Momentum
# Optimize: f(x, y) = x^2 + 10*y^2 (ravine along x-axis)

def f_ravine(x, y):
    return x**2 + 10*y**2

def grad_ravine(x, y):
    return np.array([2*x, 20*y])

# Vanilla GD
def vanilla_gd(x0, y0, lr, num_steps):
    pos = np.array([x0, y0])
    history = [pos.copy()]
    
    for _ in range(num_steps):
        grad = grad_ravine(pos[0], pos[1])
        pos = pos - lr * grad
        history.append(pos.copy())
    
    return np.array(history)

# Momentum
def momentum_gd(x0, y0, lr, beta, num_steps):
    pos = np.array([x0, y0])
    velocity = np.zeros(2)
    history = [pos.copy()]
    
    for _ in range(num_steps):
        grad = grad_ravine(pos[0], pos[1])
        velocity = beta * velocity + (1 - beta) * grad
        pos = pos - lr * velocity
        history.append(pos.copy())
    
    return np.array(history)

# Run both
x0, y0 = 5.0, 2.0
lr = 0.1
num_steps = 50

history_vanilla = vanilla_gd(x0, y0, lr, num_steps)
history_momentum = momentum_gd(x0, y0, lr, beta=0.9, num_steps)

# Visualize
x = np.linspace(-6, 6, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
Z = f_ravine(X, Y)

plt.figure(figsize=(14, 6))

# Contour plot
plt.subplot(1, 2, 1)
plt.contour(X, Y, Z, levels=30, alpha=0.6)
plt.plot(history_vanilla[:, 0], history_vanilla[:, 1], 'ro-', 
         markersize=4, linewidth=2, alpha=0.7, label='Vanilla GD')
plt.plot(history_momentum[:, 0], history_momentum[:, 1], 'bs-', 
         markersize=4, linewidth=2, alpha=0.7, label='Momentum')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Optimization Paths (Ravine Function)')
plt.legend()
plt.grid(True, alpha=0.3)

# Loss curves
plt.subplot(1, 2, 2)
losses_vanilla = [f_ravine(h[0], h[1]) for h in history_vanilla]
losses_momentum = [f_ravine(h[0], h[1]) for h in history_momentum]
plt.plot(losses_vanilla, 'ro-', linewidth=2, alpha=0.7, label='Vanilla GD')
plt.plot(losses_momentum, 'bs-', linewidth=2, alpha=0.7, label='Momentum')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Convergence Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

plt.tight_layout()
plt.show()

print(f"Final loss (Vanilla GD): {losses_vanilla[-1]:.6f}")
print(f"Final loss (Momentum): {losses_momentum[-1]:.6f}")
print("\nMomentum converges faster!")

## 5. Adam: The Most Popular Optimizer

**Adam** (Adaptive Moment Estimation) combines:
- Momentum (first moment)
- Adaptive learning rates (second moment)

### Algorithm:

Initialize:
- $m_0 = 0$ (first moment)
- $v_0 = 0$ (second moment)

For each iteration $t$:
1. Compute gradient: $g_t$
2. Update first moment: $m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t$
3. Update second moment: $v_t = \beta_2 v_{t-1} + (1-\beta_2) g_t^2$
4. Bias correction: $\hat{m}_t = \frac{m_t}{1-\beta_1^t}$, $\hat{v}_t = \frac{v_t}{1-\beta_2^t}$
5. Update parameters: $\theta_t = \theta_{t-1} - \alpha \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$

**Default hyperparameters:**
- $\beta_1 = 0.9$ (momentum)
- $\beta_2 = 0.999$ (variance)
- $\epsilon = 10^{-8}$ (numerical stability)
- $\alpha = 0.001$ (learning rate)

In [ ]:
class AdamOptimizer:
    """
    Adam optimizer implementation from scratch
    """
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None  # first moment
        self.v = None  # second moment
        self.t = 0     # time step
    
    def step(self, params, grads):
        """
        Update parameters using Adam
        
        Args:
            params: dict of parameters
            grads: dict of gradients
        """
        if self.m is None:
            # Initialize moment estimates
            self.m = {k: np.zeros_like(v) for k, v in params.items()}
            self.v = {k: np.zeros_like(v) for k, v in params.items()}
        
        self.t += 1
        
        for key in params.keys():
            # Update biased first moment
            self.m[key] = self.beta1 * self.m[key] + (1 - self.beta1) * grads[key]
            
            # Update biased second moment
            self.v[key] = self.beta2 * self.v[key] + (1 - self.beta2) * (grads[key]**2)
            
            # Bias correction
            m_hat = self.m[key] / (1 - self.beta1**self.t)
            v_hat = self.v[key] / (1 - self.beta2**self.t)
            
            # Update parameters
            params[key] -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)

print("Adam optimizer implemented!")

## 6. Comparing Optimizers on a Real Problem

In [ ]:
# Generate dataset
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

# Define model
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Train with different optimizers
def train_model(optimizer_name, lr=0.01, num_epochs=100):
    model = SimpleNet()
    criterion = nn.CrossEntropyLoss()
    
    # Create optimizer
    if optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr)
    elif optimizer_name == 'Momentum':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    
    losses = []
    accuracies = []
    
    for epoch in range(num_epochs):
        # Forward
        outputs = model(X_train_t)
        loss = criterion(outputs, y_train_t)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track metrics
        losses.append(loss.item())
        
        with torch.no_grad():
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == y_train_t).float().mean().item()
            accuracies.append(accuracy)
    
    return losses, accuracies

# Compare optimizers
optimizers = ['SGD', 'Momentum', 'Adam', 'RMSprop']
results = {}

for opt in optimizers:
    print(f"Training with {opt}...")
    losses, accs = train_model(opt, lr=0.01, num_epochs=200)
    results[opt] = {'losses': losses, 'accuracies': accs}

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for opt in optimizers:
    ax1.plot(results[opt]['losses'], label=opt, linewidth=2)
    ax2.plot(results[opt]['accuracies'], label=opt, linewidth=2)

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training Accuracy Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("- Adam typically converges fastest")
print("- Momentum helps vanilla SGD significantly")
print("- RMSprop also adapts learning rates per parameter")

## 7. Learning Rate Schedules

Often beneficial to **decrease learning rate** over time:
- Start with large steps (explore quickly)
- End with small steps (fine-tune)

### Common Schedules:

1. **Step Decay**: Reduce by factor every N epochs
   - $\alpha_t = \alpha_0 \cdot \gamma^{\lfloor t/N \rfloor}$

2. **Exponential Decay**: Smooth exponential decrease
   - $\alpha_t = \alpha_0 \cdot e^{-kt}$

3. **Cosine Annealing**: Follows cosine curve
   - $\alpha_t = \alpha_{min} + \frac{1}{2}(\alpha_{max} - \alpha_{min})(1 + \cos(\frac{t\pi}{T}))$

4. **Reduce on Plateau**: Reduce when validation loss stops improving

In [ ]:
# Visualize different schedules
num_epochs = 100
lr_0 = 0.1

# Step decay
step_lr = [lr_0 * (0.5 ** (t // 20)) for t in range(num_epochs)]

# Exponential decay
exp_lr = [lr_0 * np.exp(-0.05 * t) for t in range(num_epochs)]

# Cosine annealing
cosine_lr = [0.001 + 0.5 * (lr_0 - 0.001) * (1 + np.cos(np.pi * t / num_epochs)) 
             for t in range(num_epochs)]

# Plot
plt.figure(figsize=(12, 5))

plt.plot(step_lr, label='Step Decay', linewidth=2)
plt.plot(exp_lr, label='Exponential Decay', linewidth=2)
plt.plot(cosine_lr, label='Cosine Annealing', linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedules')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Example: Using PyTorch schedulers
model = SimpleNet()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Cosine annealing scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0.001)

# Track learning rate
lrs = []
for epoch in range(100):
    # Training step would go here
    lrs.append(optimizer.param_groups[0]['lr'])
    scheduler.step()

plt.figure(figsize=(10, 4))
plt.plot(lrs, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('PyTorch CosineAnnealingLR Scheduler')
plt.grid(True, alpha=0.3)
plt.show()

## 8. Regularization: Preventing Overfitting

### 8.1 L2 Regularization (Weight Decay)

Add penalty for large weights:

$L_{total} = L_{data} + \frac{\lambda}{2} \sum_{i} w_i^2$

Gradient becomes: $\nabla L = \nabla L_{data} + \lambda w$

**Effect**: Prevents weights from becoming too large, improves generalization

In [ ]:
# Compare with and without weight decay
def train_with_regularization(weight_decay=0.0, num_epochs=200):
    model = SimpleNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=weight_decay)
    
    train_losses = []
    test_losses = []
    
    for epoch in range(num_epochs):
        # Train
        outputs = model(X_train_t)
        loss = criterion(outputs, y_train_t)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        # Test
        with torch.no_grad():
            test_outputs = model(X_test_t)
            test_loss = criterion(test_outputs, y_test_t)
            test_losses.append(test_loss.item())
    
    return train_losses, test_losses

# Train with different weight decay values
wd_values = [0.0, 0.001, 0.01]
results_wd = {}

for wd in wd_values:
    print(f"Training with weight_decay={wd}...")
    train_loss, test_loss = train_with_regularization(weight_decay=wd)
    results_wd[wd] = {'train': train_loss, 'test': test_loss}

# Plot
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
for wd in wd_values:
    plt.plot(results_wd[wd]['train'], label=f'WD={wd}', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss vs Weight Decay')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
for wd in wd_values:
    plt.plot(results_wd[wd]['test'], label=f'WD={wd}', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Test Loss')
plt.title('Test Loss vs Weight Decay')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservation: Moderate weight decay often improves test performance!")

## Summary

### Key Optimization Concepts:

1. **Gradient Descent**
   - Update: $\theta \leftarrow \theta - \alpha \nabla L$
   - Goes "downhill" to minimize loss
   - Three variants: Batch, Stochastic, Mini-batch

2. **Learning Rate**
   - Most important hyperparameter
   - Too small = slow, too large = unstable
   - Typical starting values: 0.001 - 0.01

3. **Momentum**
   - Accelerates in consistent directions
   - Dampens oscillations
   - Default: $\beta = 0.9$

4. **Adam** (Recommended Default)
   - Combines momentum + adaptive learning rates
   - Works well out of the box
   - Defaults: $\alpha=0.001, \beta_1=0.9, \beta_2=0.999$

5. **Learning Rate Schedules**
   - Reduce LR over time for fine-tuning
   - Common: Step decay, cosine annealing

6. **Regularization**
   - L2 (weight decay): Prevents large weights
   - Improves generalization

### Practical Recommendations:

**For most problems:**
- Start with Adam optimizer
- Learning rate: 0.001 (or tune)
- Mini-batch size: 32-128
- Add weight decay: 0.001-0.01

**If Adam doesn't work:**
- Try SGD with momentum (0.9)
- Use learning rate scheduler
- Tune learning rate carefully

### Next Steps:
In the next notebook, we'll explore **attention mechanisms** - the foundation of modern transformers!

## Practice Exercises

In [ ]:
# Exercise 1: Implement RMSprop from scratch
# RMSprop: v_t = beta * v_{t-1} + (1-beta) * g_t^2
#          theta_t = theta_{t-1} - alpha * g_t / sqrt(v_t + epsilon)
# Your code here


In [ ]:
# Exercise 2: Find the optimal learning rate
# Use learning rate finder: train with exponentially increasing LRs
# Plot loss vs LR to find the sweet spot
# Your code here


In [ ]:
# Exercise 3: Implement early stopping
# Stop training when validation loss stops improving for N epochs
# Your code here
